In [5]:
import pandas as pd
import re
import json

In [6]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
with open("../data/bronze/Details.json", 'r') as json_file:
    data = json.load(json_file)

In [ ]:
data

## FUNCTIONS

In [8]:
def get_number(text: str):
    cleaned = re.findall(r'\d+', text)[0]
    return int(cleaned)

In [32]:
def get_header(parent_container):
    information = {}
    try:
        card_header = parent_container.find_element(By.CLASS_NAME, "card-header")
        header_text = card_header.find_element(By.CLASS_NAME, "details").text
        header_date = card_header.find_element(By.CLASS_NAME, "date").text
        header_condition = parent_container.find_element(By.TAG_NAME, "strong").text

        information = {
            "Information": header_text,
            "Date": header_date,
            "Condition": header_condition
        }
    except Exception as e:
        print("Error when getting information details")
    finally:
        return information

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
waitTime = WebDriverWait(driver,10)
reviews = {}

for supplement, information in data.items():
    numReviews = get_number(information["Reviews"]['NumberOfReviews'])
    url = information["Reviews"]['Url']

    if numReviews == 0:
        continue

    try:
        driver.get(url)
        
        container = waitTime.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".reviews-page .shared-reviews-container")))
        ul_pagination = container.find_element(By.CSS_SELECTOR, ".pagination-holder .pagination-holder .pagination")
        
        li_items = ul_pagination.find_elements(By.TAG_NAME, 'li')

        review_card_holder = container.find_elements(By.CSS_SELECTOR, ".review-details-holder")
        details = []
        for holder in review_card_holder:
            
            review_details = holder.find_element(By.CSS_SELECTOR, ".review-details")

            headers = get_header(review_details)
            description = get_description()

            details.append({ 
                headers
            })

        reviews[supplement] = details

    except Exception as e:
        print(e)
        continue

driver.quit()

unhashable type: 'dict'


In [34]:
reviews

{'Aloe': [{'Header': {'Information': 'ReflectiveGypsy | Age: 35-44 | On supplement for 10 years or more |',
    'Date': '11/22/2014',
    'Condition': 'Condition: Burns'}}]}